### 가치지표 계산

In [3]:
import keyring
import requests as rq
from io import BytesIO
import zipfile
import xmltodict
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time

In [4]:
ip = '43.201.116.164'
engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='12345678',
                      host=f'{ip}',
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()

In [9]:
from sqlalchemy import create_engine
import pandas as pd


# 티커 리스트
ticker_list = pd.read_sql("""
                          select * from kor_ticker
                          where 기준일 = (select max(기준일) from kor_ticker) and 종목구분 = '보통주';
                          """, con=engine)

# 삼성전자 분기 재무제표
sample_fs = pd.read_sql("""
                        select * from fs_data
                        where 공시구분 = 'q'
                        and 종목코드 = '005930'
                        and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
                        """, con=engine)
sample_fs

engine.dispose()

sample_fs = sample_fs.sort_values(['종목코드', '계정', '기준일'])
sample_fs['ttm'] = sample_fs.groupby(
    ['종목코드', '계정'], as_index=False)['값'].rolling(window=4, min_periods=4).sum()['값']
sample_fs

# 클랜징 -> 자본같은경우 합칠필요 없음, 필요없는 부분들 삭제
import numpy as np

sample_fs['ttm'] = np.where(sample_fs['계정'] == '자본', sample_fs['ttm'] / 4, sample_fs['ttm'])

sample_fs = sample_fs.groupby(['계정', '종목코드']).tail(1)
sample_fs

sample_fs_merge = sample_fs[['계정', '종목코드', 'ttm']].merge(
    ticker_list[['종목코드', '시가총액', '기준일']], on='종목코드')
sample_fs_merge

## 단위 맞춰주기. (원, 억원)
sample_fs_merge['시가총액'] = sample_fs_merge['시가총액'] / 100000000

# 가치지표 계산
sample_fs_merge['value'] = sample_fs_merge['시가총액'] / sample_fs_merge['ttm']
sample_fs_merge['지표'] = np.where(
    sample_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        sample_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(
            sample_fs_merge['계정'] == '자본', 'PBR',
            np.where(sample_fs_merge['계정'] == '당기순이익', 'PER', None))))

sample_fs_merge

# 배당수익률 계산
ticker_list_sample = ticker_list[ticker_list['종목코드'] == '005930'].copy()
ticker_list_sample['DY'] = ticker_list_sample['주당배당금'] / ticker_list_sample['종가']
ticker_list_sample


,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분,index,DY
366,005930,삼성전자,KOSPI,69800.0,4.166910e+14,2023-09-19,8057.0,3887.0,57822.0,1444.0,보통주,445,0.020688


In [13]:
from sqlalchemy import create_engine
ip = '43.201.116.164'
engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='12345678',
                      host=f'{ip}',
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()


# 전종목 가치지표 계산

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np


# 분기 재무제표 불러오기
kor_fs = pd.read_sql("""
                     select * from fs_data
                     where 공시구분 = 'q' 
                     and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
                     """, con=engine)


# 티커 리스트 불러오기
ticker_list = pd.read_sql("""
                    select * from kor_ticker
                    where 기준일 = (select max(기준일) from kor_ticker) and 종목구분 = '보통주';
                    """, con=engine)

engine.dispose()

# TTM 계산
kor_fs = kor_fs.sort_values(['종목코드', '계정', '기준일'])
kor_fs['ttm'] = kor_fs.groupby(['종목코드', '계정'], as_index=False)['값'].rolling(
    window=4, min_periods=4).sum()['값']

# 자본은 평균으로
kor_fs['ttm'] = np.where(kor_fs['계정'] == '자본', kor_fs['ttm'] / 4, kor_fs['ttm'])
kor_fs = kor_fs.groupby(['계정', '종목코드']).tail(1)


# 가치지표 계산
kor_fs_merge = kor_fs[['계정', '종목코드', 'ttm']].merge(
    ticker_list[['종목코드', '시가총액', '기준일']], on='종목코드')

kor_fs_merge['시가총액'] = kor_fs_merge['시가총액'] / 100000000

kor_fs_merge['value'] = kor_fs_merge['시가총액'] / kor_fs_merge['ttm']
kor_fs_merge['value'] = kor_fs_merge['value'].round(4)
kor_fs_merge['지표'] = np.where(
    kor_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        kor_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(
            kor_fs_merge['계정'] == '자본', 'PBR',
            np.where(kor_fs_merge['계정'] == '당기순이익', 'PER', None))))

# inf 데이터 (분모가 0이라) None으로 변경
kor_fs_merge.rename(columns={'value': '값'}, inplace=True)
kor_fs_merge = kor_fs_merge[['종목코드', '기준일', '지표', '값']]
kor_fs_merge = kor_fs_merge.replace([np.inf, -np.inf, np.nan], None)


# sql에 저장
# query = """
#     insert into kor_value (종목코드, 기준일, 지표, 값)
#     values (%s,%s,%s,%s) as new
#     on duplicate key update
#     값=new.값
# """
dy_list.to_sql(name='kor_value', con=engine, index=True, if_exists='append')
# args_fs = kor_fs_merge.values.tolist()
# mycursor.executemany(query, args_fs)
# con.commit()

# 배당 수익률 계산
ticker_list['값'] = ticker_list['주당배당금'] / ticker_list['종가']
ticker_list['값'] = ticker_list['값'].round(4)
ticker_list['지표'] = 'DY'
dy_list = ticker_list[['종목코드', '기준일', '지표', '값']]
dy_list = dy_list.replace([np.inf, -np.inf, np.nan], None)
dy_list = dy_list[dy_list['값'] != 0]

dy_list.to_sql(name='kor_value', con=engine, index=True, if_exists='append')

# args_dy = dy_list.values.tolist()
# mycursor.executemany(query, args_dy)
# con.commit()

# engine.dispose()
# con.close()

1169

In [19]:
dy_list

,종목코드,기준일,지표,값
0,095570,2023-09-19,DY,0.0605
1,006840,2023-09-19,DY,0.0107
2,027410,2023-09-19,DY,0.0303
3,282330,2023-09-19,DY,0.0275
4,138930,2023-09-19,DY,0.0867
...,...,...,...,...
2366,084110,2023-09-19,DY,0.0209
2368,024060,2023-09-19,DY,0.0183
2369,010240,2023-09-19,DY,0.0332
2370,189980,2023-09-19,DY,0.0172


In [18]:
kor_fs_merge.to_sql(name='kor_value', con=engine, index=True, if_exists='append')

9368